In [1]:
import requests
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

In [2]:
CLIENT_ID = '4a369d6bbccb492e9a425211173f2feb'
CLIENT_SECRET = '162a44fc5ac4438c829dcd0416c7bfe4'
client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) #spotify object to access API

In [3]:
AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

In [4]:
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

In [5]:
BASE_URL = 'https://api.spotify.com/v1/'

def get_audio_features(artist, song):
    sres = sp.search(q="artist:" + artist + " track:" + song, type="track")
    # no result found
    if not sres['tracks']['items']:
        return None
    else:
        track_id = sres['tracks']['items'][0]['uri'].split(':')[-1]
        req = requests.get(BASE_URL + 'audio-features/' + track_id, headers=headers)
        return req.json()

In [6]:
# collect audio features for tracks we have lyrics available for
charts_path = "../data/interim/lyrics-meta/"

for year in range(2005,2021):
    in_path = charts_path+"lyrics_bb_hot_100_"+str(year)+".csv"
    data = pd.read_csv(in_path)
    
    d = []
    for index, row in data.iterrows():
        # try getting audio features
        if get_audio_features(row["artist"], row["song"]) is not None:
            afeat_dict = get_audio_features(row["artist"], row["song"])
            # print(afeat_dict) {'error': {'status': 404, 'message': 'analysis not found'}}
            # should be handled in dunction above
            if 'error' in afeat_dict.keys():
                if afeat_dict['error']['status'] == 404:
                    continue
                else:
                    print(afeat_dict['error'])
        
            d.append({
                "artist" : row["artist"],
                "song" : row["song"],
                "lyricId" : row["lyricId"],
                "danceability": afeat_dict["danceability"],
                "energy": afeat_dict["energy"],
                "key": afeat_dict["key"],
                "loudness": afeat_dict["loudness"],
                "mode": afeat_dict["mode"],
                "speechiness": afeat_dict["speechiness"],
                "acousticness": afeat_dict["acousticness"],
                "instrumentalness": afeat_dict["instrumentalness"],
                "liveness": afeat_dict["liveness"],
                "valence": afeat_dict["valence"],
                "tempo": afeat_dict["tempo"],
                "uri": afeat_dict["uri"],
                "duration_ms": afeat_dict["duration_ms"]
            })
    
    afeat_data = pd.DataFrame(d)
    out_path = "../data/interim/audio-feat/afeat_bb_hot_100_" + str(year) + ".csv"
    afeat_data.to_csv(out_path, index=False)
    print("year done: {}".format(year))

year done: 2005
year done: 2006
year done: 2007
year done: 2008
year done: 2009
year done: 2010
year done: 2011
year done: 2012
year done: 2013
year done: 2014
year done: 2015
year done: 2016
year done: 2017
year done: 2018
year done: 2019
year done: 2020
